In [1]:
from numba import cuda

@cuda.jit
def matmul(A,B,C):
    # C=A *B
    i,j=cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        tmp = 0
        for k in range(A.shape[1]):
            tmp+=A[i,k]*B[k,j]
        C[i,j]=tmp


In [2]:
import numpy as np
A=np.asarray([[0.3,0],[0,0.5]])
B=np.asarray([[0.,0.2],[0.1,0.]])
C=B.copy()
matmul(A,B,C)
print(C)

CudaDriverError: driver missing function: cuInit.
Requires CUDA 7.5 or above.
